<a href="https://colab.research.google.com/github/Sim-mi-gyeong/DeepLearning/blob/main/pytorch_%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EB%AA%A8%EB%8D%B8_%EA%B5%AC%EC%84%B1%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 신경망: data 연산 수행하는 계층(layer)/모듈(module)로 구성
# torch.nn 네임스페이스: 신경망 구성에 필요한 모든 구성요소 제공
# Pytorch의 모든 모듈은 nn.Module의 하위 클래스(subclass)
# 신경망은 다른 모듈(계층,layer)로 구성된 모듈

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
# 학습을 위한 장치 얻기
# torch.cuda를 사용할 수 있는지 확인 -> GPU 사용 or CPU 계속 사용

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [ ]:
# 클래스 정의하기
# 신경망 모델을 nn.Module의 하위클래스로 정의
# __init__에서 신경망 계층 초기화
# nn.Module을 상속받은 모든 class는 forward 메소드에 입력 데이터에 대한 연산 구현 

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [ ]:
# NeuralNetwork의 인스턴스(instance) 생성 -> device로 이동한 후 -> 구조(structure) 출력

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [ ]:
# 모델을 사용하기 위해 입력 데이터 전달 -> 일부 백그라운드 연산들과 함께 모델의 forward 실행
# model.forward() 직접 호출 X!!

# 모델에 입력 호출 -> 각 분류(class)에 대한 원시(raw) 예측값이 있는 10-차원 텐서 반환
# 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률 얻음

X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
print(f'Predicted Probability : {pred_probab}')
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted Probability : tensor([[0.1101, 0.0975, 0.0975, 0.1016, 0.0975, 0.1014, 0.0975, 0.0975, 0.0975,
         0.1021]], grad_fn=<SoftmaxBackward>)
Predicted class: tensor([0])


In [ ]:
# 모델 계층(Layer)
# FashionMNIST 모델의 계층 -> 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와 신경망 통과시 발생하는 일

input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [ ]:
# nn.Flatten
# nn.Flatten 계층을 초기화하여 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환
# dim=0의 미니배치 차원은 유지

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [ ]:
# nn.Linear
# 선형 계층은 저장된 가중치(weight)와 편향(bias)을 사용해 입력에 선형 변환 적용하는 모듈

layer1 = nn.Linear(in_features = 28*28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [ ]:
# nn.ReLU
# 비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping) 형성
# 선형 변환 후에 적용 -> 비선형성 도입, 신경망이 다양한 현상 학습 할 수 있도록

print(f'Before ReLU : {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU : {hidden1}')  # 음수는 0으로

Before ReLU : tensor([[-0.3244, -0.5843,  0.2743, -0.3958, -0.1436,  0.2623, -0.2406, -0.0793,
         -0.2273, -0.2069, -0.2645,  0.0355,  0.1394,  0.3257, -0.5710,  0.3705,
         -0.1852,  0.0089, -0.2807,  0.0008],
        [-0.1375, -0.5746,  0.6205, -0.3534, -0.2366,  0.6378, -0.0776,  0.1338,
          0.0163, -0.0669,  0.1819, -0.4583,  0.5387,  0.4415, -0.7350,  0.2107,
         -0.1022, -0.2008, -0.4738, -0.1376],
        [-0.1870, -0.2258,  0.1088, -0.0010, -0.3252,  0.1823,  0.0930, -0.1759,
          0.0123,  0.0993, -0.0219, -0.3154, -0.0565,  0.2526, -0.4051,  0.4391,
          0.2720, -0.1467, -0.7402, -0.0857]], grad_fn=<AddmmBackward>)


After ReLU : tensor([[0.0000, 0.0000, 0.2743, 0.0000, 0.0000, 0.2623, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0355, 0.1394, 0.3257, 0.0000, 0.3705, 0.0000, 0.0089,
         0.0000, 0.0008],
        [0.0000, 0.0000, 0.6205, 0.0000, 0.0000, 0.6378, 0.0000, 0.1338, 0.0163,
         0.0000, 0.1819, 0.0000, 0.5387, 0.4415, 0.0

In [ ]:
 # nn.Seauential
 # 순서를 갖는 모듈의 컨테이너
 # data는 정의된 순서로 모든 모듈을 통해 전달됨

 seq_modules = nn.Sequential(
     flatten,
     layer1,
     nn.ReLU(),
     nn.Linear(20, 10)
 )
 input_image = torch.rand(3, 28, 28)
 logits = seq_modules(input_image)
 print(logits)
 print(logits.size())

tensor([[-0.0539,  0.1216, -0.2975,  0.1631,  0.2873,  0.0842,  0.0521,  0.0255,
         -0.1463,  0.1449],
        [ 0.0287,  0.1107, -0.2793,  0.1868,  0.2885,  0.1934, -0.0455,  0.0629,
         -0.1611,  0.0880],
        [ 0.0468,  0.0762, -0.3320,  0.2090,  0.3070,  0.1668, -0.0164,  0.0915,
         -0.1137,  0.1474]], grad_fn=<AddmmBackward>)
torch.Size([3, 10])


In [ ]:
# nn.Softmax
# 신경망의 마지막 선형 계층: nn.Softmax 모듈에 전달될 ([-infinty, infinity] 범위의 원시 값(raw value)인) logits 반환
# logits: 모델의 각 분류(class)에 대한 예측확률을 나타내도록 [0,1] 범위로 비례하여 조정(scale)됨.
# dim 매개변수는 값의 합이 1이 되는 차원을 나타냄

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

print(pred_probab)

tensor([[0.0901, 0.1074, 0.0706, 0.1119, 0.1267, 0.1034, 0.1002, 0.0975, 0.0822,
         0.1099],
        [0.0969, 0.1052, 0.0712, 0.1135, 0.1257, 0.1143, 0.0900, 0.1003, 0.0802,
         0.1028],
        [0.0975, 0.1004, 0.0668, 0.1147, 0.1265, 0.1099, 0.0915, 0.1019, 0.0830,
         0.1078]], grad_fn=<SoftmaxBackward>)


In [ ]:
# 모델 매개변수
# 신경망 내부의 계층들은 매개변수화(parameterize)된다. -> 학습 중에 최적화되는 가중치와 편향과 연관지어짐
# nn.Module 상속 -> 모델 객체 내부의 모든 필드들이 자동으로 추적/
#                   모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근 가능

# 각 매개변수들을 순회(iterate)하며 매개변수의 크기와 값을 출력
print("Model structure: ", model, "\n\n")
for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")
  print(param)

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0135, -0.0090,  0.0052,  ..., -0.0090, -0.0347,  0.0172],
        [ 0.0166,  0.0276,  0.0224,  ...,  0.0078,  0.0200, -0.0235]],
       grad_fn=<SliceBackward>) 

Parameter containing:
tensor([[ 0.0135, -0.0090,  0.0052,  ..., -0.0090, -0.0347,  0.0172],
        [ 0.0166,  0.0276,  0.0224,  ...,  0.0078,  0.0200, -0.0235],
        [-0.0327,  0.0132,  0.0345,  ...,  0.0205,  0.0224,  0.0004],
        ...,
        [-0.0164,  0.0221, -0.0347,  ..., -0.0269, -0.0081,  0.0336],
        [-0.0198, -0.0079, -0.0058,  ..., -0.0146, -0.0019,  0.0240],
        [